The goal of this notebook is to identify possible ways of collecting this data.

# Available repositories

## mwparserfromhell

In [1]:
import mwapi
session = mwapi.Session('https://en.wikipedia.org', user_agent='williamvoje@gmail.com')


In [2]:
page_request_data = session.get(action='query', titles ='git', prop = 'revisions', rvprop='content')
page_request_data['query']['pages']['1771747']['revisions'][0]['contentmodel']

'wikitext'

In [30]:
possible_page_data = page_request_data['query']['pages']['1771747']['revisions'][0]['*']

In [31]:
import mwparserfromhell

In [33]:
parsed = mwparserfromhell.parse(res["query"]["pages"]['1771747']['revisions'][0]['*'])

NameError: name 'res' is not defined

In [ ]:
parsed.filter_wikilinks()

In [ ]:
parsed.filter_headings()

In [ ]:
parsed.get_sections()[0].filter_wikilinks()

In [34]:
len(parsed.strip_code())

NameError: name 'parsed' is not defined

Looks like mwparserfromhell will get the job done for parsing the data of raw information

In [35]:
page_request_data = session.get(action='query', titles ='git', prop = 'revisions', rvprop='content')
page_request_data['query']['pages']['1771747']['revisions'][0]['contentmodel']

'wikitext'

In [11]:
page_request_data = session.get(action='query', titles ='Git', prop = 'revisions', 
                                rvprop='ids|flags|timestamp|comment|user|userid|content',
                                rvlimit='max', rvstop='2015-07-01T11:03:28Z')

The following query raised warnings: {'action': 'query', 'titles': 'Git', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'rvstop': '2015-07-01T11:03:28Z', 'format': 'json'}
	- main -- {'*': 'Unrecognized parameter: rvstop.'}


In [13]:
next(iter(page_request_data['query']['pages'].values()))['revisions']

[{'revid': 847024552,
  'parentid': 847014199,
  'user': 'Keith D',
  'userid': 2278355,
  'timestamp': '2018-06-22T11:31:59Z',
  'comment': 'Fix date',
  'contentformat': 'text/x-wiki',
  'contentmodel': 'wikitext',
  '*': '{{Other uses|Git (disambiguation)}}\n\n{{Use dmy dates|date=March 2012}}\n\n{{Infobox software\n| name = Git\n| logo = [[File:Git-logo-2012.svg|150px]]\n| screenshot = Git session.png\n| caption = A command-line session showing repository creation, addition of a file, and remote synchronization\n| author = [[Linus Torvalds]]<ref name="git-initial-commit">{{cite web |url = https://github.com/git/git/commit/e83c5163316f89bfbde7d9ab23ca2e25604af290 |date = 8 April 2005 |title = Initial revision of "git", the information manager from hell |publisher = Github |accessdate = 20 December 2015 |deadurl = no |archiveurl = https://web.archive.org/web/20151116175401/https://github.com/git/git/commit/e83c5163316f89bfbde7d9ab23ca2e25604af290 |archivedate = 16 November 2015 |df =

In [82]:
%%timeit
session.get(action='query', titles ='git', prop = 'revisions', 
                    rvprop='ids|flags|timestamp|comment|user|userid|content',
                    rvlimit='max')

851 ms ± 68.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Looks like we have all of the tools we need to start collecting this data I'm going to test this on a small page


First let's build out the tools we need to extract the data

In [87]:
test_page = session.get(action='query', titles ='The_Faint', prop = 'revisions', 
                                rvprop='ids|flags|timestamp|comment|user|userid|content',
                                rvlimit='max')

In [102]:
test_page['query']['pages']

{'168318': {'pageid': 168318,
  'ns': 0,
  'title': 'The Faint',
  'revisions': [{'revid': 833190861,
    'parentid': 817750104,
    'user': '70.178.162.127',
    'anon': '',
    'userid': 0,
    'timestamp': '2018-03-30T04:32:15Z',
    'comment': '/* Vverevvolf Grehv */',
    'contentformat': 'text/x-wiki',
    'contentmodel': 'wikitext',
    '*': '{{refimprove|date=June 2016}}\n{{Infobox musical artist\n| name             = The Faint\n| image            = TheFaintLive2008-01.jpg\n| caption          = The Faint live on tour 2008\n| background       = group_or_band\n| origin           = [[Omaha, Nebraska]]\n| genre            = [[Indie rock]], [[post-punk revival]], [[dance-punk]], [[New wave music|new wave]]\n| years_active     = 1995&ndash;present\n| label            = Blank.wav, [[Saddle Creek]], [[Astralwerks]]\n| associated_acts  = [[Vverevvolf Grehv]], [[Broken Spindles]], Depressed Buttons, [[Digital Leather]], TIT, [[Reptar (band)]]\n| website          = {{URL|www.thefaint.com}

In [26]:
import pandas as pd
import os

In [43]:
# This assumes that you are only looking at single page

page_title = 'The_Faint'
path_to_data = '../data/20180618_Test_history_data/'

headings = ['revid', 'parentid', 'user', 'userid', 'timestamp', 'comment', 
            'character_count', 'external_link_count', 'heading_count', 'wikilink_count', 'wikilinks']

for rev_count, revision_set in enumerate(session.get(continuation=True, action='query', titles =page_title, prop = 'revisions', 
                                rvprop='ids|flags|timestamp|comment|user|userid|content',
                                rvlimit='max')):
    
    # Make dataframe for storing information
    
    data_frame = pd.DataFrame(columns=headings)
    

    for count, revision in enumerate(next(iter(revision_set['query']['pages'].values()))['revisions']):

        # Extract edit information 
        revid, parentid, user, userid, timestamp, comment = revision_information(revision)

        # Extract information on the website itself
        parsed = mwparserfromhell.parse(revision['*'])

        character_count, external_link_count, heading_count, wikilink_count, wikilinks = parsed_article_metrics(parsed)
        
        data_frame.loc[count] = [ revid, parentid, user, userid, timestamp, comment,
                                 character_count, external_link_count, heading_count, wikilink_count, wikilinks]
        
    # Dump the dataframe
    data_frame.to_csv(os.path.join(path_to_data, page_title + "_" + str(rev_count) + '.csv'))
        
#         print(character_count, external_link_count, heading_count, wikilink_count)

KeyboardInterrupt: 

In [38]:
data_frame['wikilinks'][0]

IndexError: index out of bounds

In [40]:
def revision_information(revision_json):
    """ This will export 
    revid, parentid, user, userid, timestamp, comment
    """
    return [revision['revid'], revision['parentid'], revision['user'], revision['userid'],
           revision['timestamp'], revision['comment']]


In [41]:
def parsed_article_metrics(parsed_article):
    """This should take a parsed article
    Presently I'm using a mwparserfromhell"""
    character_count = len(parsed_article.strip_code())
    external_link_count = len(parsed_article.filter_external_links())
    heading_count = len(parsed_article.filter_headings())
    wikilink_count = len(set([str(link) for link in parsed_article.filter_wikilinks()]))            
    
    return character_count, external_link_count, heading_count, wikilink_count, parsed.filter_wikilinks()
    
    

In [121]:
revid, parentid, user, userid, annon, timestamp, comment = revision_information(revision)

[501689658, 501689449, '64.24.35.16', '', '2012-07-11T08:51:28Z', '']

In [143]:
len(set([str(link) for link in parsed.filter_wikilinks()]))

76

In [45]:
parsed.filter_wikilinks()

['[[Omaha, Nebraska]]',
 '[[Indie rock]]',
 '[[post-punk revival]]',
 '[[dance-punk]]',
 '[[New wave music|new wave]]',
 '[[Saddle Creek]]',
 '[[Astralwerks]]',
 '[[Vverevvolf Grehv]]',
 '[[Broken Spindles]]',
 '[[Digital Leather]]',
 '[[Todd Fink]]',
 '[[Jacob Thiele]]',
 '[[Dapose]]',
 '[[Clark Baechle]]',
 '[[Conor Oberst]]',
 '[[Joel Petersen]]',
 '[[indie rock]]',
 '[[Omaha, Nebraska]]',
 '[[Todd Fink]]',
 '[[Jacob Thiele]]',
 '[[Joel Petersen]]',
 '[[Clark Baechle]]',
 '[[Conor Oberst]]',
 '[[Bright Eyes (band)|Bright Eyes]]',
 '[[Saddle Creek Records]]',
 '[[Skateboarding|skateboarded]]',
 '[[Clark Baechle]]',
 '[[Todd Fink]]',
 '[[Joel Petersen]]',
 '[[Saddle Creek Records]]',
 '[[Media (album)|Media]]',
 '[[Jacob Thiele]]',
 '[[United States|U.S.]]',
 '[[Blank-Wave Arcade]]',
 '[[Danse Macabre (album)|Danse Macabre]]',
 '[[death metal]]',
 '[[Mike Mogis]]',
 '[[Wet from Birth]]',
 '[[Saddle Creek]]',
 '[[Fasciinatiion]]',
 '[[Boys Noize]]',
 '[[Ladytron]]',
 '[[idiomag]]',
 '[

In [135]:
link = links[0]

In [157]:
parsed.filter_wikilinks()[0].ipynb_checkpoints/

'[[Omaha, Nebraska]]'

In [159]:
link.title

'Omaha, Nebraska'

In [ ]:
# process to complete

# init the process with the webpage 
session = mwapi.Session('https://en.wikipedia.org', user_agent='williamvoje@gmail.com')

# iterate through page revision history
for page_revisions in 





In [4]:
import pandas as pd
import os

In [14]:
# Consolidating all csvs in directory 
path_to_data = '../data/20180618_Test_history_data/'


df = pd.concat(pd.read_csv(
                
                for f in os.listdir(path_to_data)))




os.path.join(path_to_data, f) for f in os.listdir(path_to_data)

ValueError: Invalid file path or buffer object type: <class 'generator'>

In [15]:
[os.path.join(path_to_data, f) for f in os.listdir(path_to_data)]

['../data/20180618_Test_history_data/The_Faint_0.csv',
 '../data/20180618_Test_history_data/The_Faint_1.csv',
 '../data/20180618_Test_history_data/The_Faint_10.csv',
 '../data/20180618_Test_history_data/The_Faint_11.csv',
 '../data/20180618_Test_history_data/The_Faint_2.csv',
 '../data/20180618_Test_history_data/The_Faint_3.csv',
 '../data/20180618_Test_history_data/The_Faint_4.csv',
 '../data/20180618_Test_history_data/The_Faint_5.csv',
 '../data/20180618_Test_history_data/The_Faint_6.csv',
 '../data/20180618_Test_history_data/The_Faint_7.csv',
 '../data/20180618_Test_history_data/The_Faint_8.csv',
 '../data/20180618_Test_history_data/The_Faint_9.csv']

In [8]:
for x in os.walk(path_to_data):
    print(x)

('../data/20180618_Test_history_data/', [], ['The_Faint_0.csv', 'The_Faint_1.csv', 'The_Faint_10.csv', 'The_Faint_11.csv', 'The_Faint_2.csv', 'The_Faint_3.csv', 'The_Faint_4.csv', 'The_Faint_5.csv', 'The_Faint_6.csv', 'The_Faint_7.csv', 'The_Faint_8.csv', 'The_Faint_9.csv'])


In [18]:
test = os.listdir(path_to_data)

In [19]:
test

['The_Faint_0.csv',
 'The_Faint_1.csv',
 'The_Faint_10.csv',
 'The_Faint_11.csv',
 'The_Faint_2.csv',
 'The_Faint_3.csv',
 'The_Faint_4.csv',
 'The_Faint_5.csv',
 'The_Faint_6.csv',
 'The_Faint_7.csv',
 'The_Faint_8.csv',
 'The_Faint_9.csv']

In [20]:
import glob

In [24]:
[i for i in glob.iglob(path_to_data + '*.csv')]

['../data/20180618_Test_history_data\\The_Faint_0.csv',
 '../data/20180618_Test_history_data\\The_Faint_1.csv',
 '../data/20180618_Test_history_data\\The_Faint_10.csv',
 '../data/20180618_Test_history_data\\The_Faint_11.csv',
 '../data/20180618_Test_history_data\\The_Faint_2.csv',
 '../data/20180618_Test_history_data\\The_Faint_3.csv',
 '../data/20180618_Test_history_data\\The_Faint_4.csv',
 '../data/20180618_Test_history_data\\The_Faint_5.csv',
 '../data/20180618_Test_history_data\\The_Faint_6.csv',
 '../data/20180618_Test_history_data\\The_Faint_7.csv',
 '../data/20180618_Test_history_data\\The_Faint_8.csv',
 '../data/20180618_Test_history_data\\The_Faint_9.csv']

In [47]:
df = pd.concat(pd.read_csv(i) for i in glob.iglob(path_to_data + '*.csv'))

In [49]:
df['wikilinks']

0     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
1     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
2     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
3     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
4     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
5     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
6     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
7     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
8     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
9     ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
10    ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
11    ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
12    ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
13    ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
14    ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
15    ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
16    ['[[Omaha, Nebraska]]', '[[Indie rock]]', '[[p...
17    ['[[Omaha, Nebraska]]', '[[Indie rock]]', 